In [1]:
import os
import concurrent.futures
import fitz
import logging
from pathlib import Path

import pandas as pd
import torch
from cleantext import clean
from datasets import load_dataset
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm.auto import tqdm
import re

# Assuming you have a TextAnalysis module
import TextAnalysis as TA

# Configure logging
logging.basicConfig(level=logging.INFO)

# --- Helper Functions ---

def remove_text_in_brackets(text):
  """Removes text enclosed in parentheses or square brackets."""
  pattern = r'\([^)]*\)|\[[^\]]*\]'
  return re.sub(pattern, '', text)

def remove_lines_starting_with_number_or_symbol(text):
  """Removes lines starting with a number or symbol."""
  pattern = r'^[\d\W].*$'
  return re.sub(pattern, '', text, flags=re.MULTILINE)

def remove_lines_with_one_word(text):
  """Removes lines containing only one word."""
  lines = text.split('\n')
  pattern = r'^\s*\w+\s*$'
  filtered_lines = [line for line in lines if not re.match(pattern, line)]
  return '\n'.join(filtered_lines)

def remove_empty_lines(text):
  """Removes empty lines."""
  lines = text.split('\n')
  non_empty_lines = [line for line in lines if line.strip() != '']
  return '\n'.join(non_empty_lines)

def find_and_remove_references(text, keywords):
  """Finds and removes text after the first occurrence of any of the given keywords."""
  earliest_position = -1
  for keyword in keywords:
    position = text.find(keyword)
    if position != -1:
      earliest_position = position if earliest_position == -1 else min(position, earliest_position)
  if earliest_position != -1:
    text = text[:earliest_position]
  return text

def process_text(text):
  """Applies a series of cleaning steps to the text."""
  keywords = ["REFERENCES", "ACKNOWLEDGEMENTS", "References", "Acknowledgements"]

  text = find_and_remove_references(text, keywords)
  text = find_and_remove_references(text, keywords)
  text = remove_text_in_brackets(text)
  text = remove_lines_starting_with_number_or_symbol(text)
  text = remove_lines_with_one_word(text)
  text = remove_empty_lines(text)

  return text

def extract_and_process_text(pdf_path, output_path):
  """Extracts text from a PDF file, cleans it, and saves it to a text file."""
  pdf_doc = fitz.open(pdf_path)
  text = ''
  for page in pdf_doc:
    text += page.get_text()

  lines = text.split('\n')
  filtered_lines = [line for line in lines if len(line.strip()) > 1]
  filtered_text = '\n'.join(filtered_lines)
  cleaned_text = process_text(filtered_text)

  with open(output_path, 'w', encoding='utf-8') as output_file:
    output_file.write(cleaned_text)
  return output_path

def process_pdf(pdf_filename):
  """Processes a single PDF file, extracts and cleans text, and returns the text filename."""
  text_filename = pdf_filename.replace(".pdf", ".txt")
  cleaned_text = extract_and_process_text(pdf_filename, text_filename)
  return cleaned_text

# --- PDF Processing ---

# Specify the directory containing your PDF files
content_directory = "C:\\Users\\admin\\Desktop\\Intern\\ReviewerExtractorFork\\ReviewerExtractor\\LLM\\content"

# Find all PDF files in the directory
pdf_filenames = [os.path.join(content_directory, file) for file in os.listdir(content_directory) if file.endswith('.pdf')]

# Process the PDF files in parallel
text_filenames = []
num_workers = 100

with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
  results = list(executor.map(process_pdf, pdf_filenames))

text_filenames.extend(result for result in results if result is not None)

# --- Summarization Model ---

hf_tag = "pszemraj/led-large-book-summary"
_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSeq2SeqLM.from_pretrained("./model").to(_device)
tokenizer = AutoTokenizer.from_pretrained("./model")
token_batch_length = 3072
batch_stride = 50
max_len_ratio = 8

settings = {
    'min_length': 5,
    'max_length': int(token_batch_length // max_len_ratio),
    'no_repeat_ngram_size': 7,
    'encoder_no_repeat_ngram_size': 7,
    'repetition_penalty': 3.7,
    'num_beams': 12,
    'length_penalty': 0.5,
    'early_stopping': True,
    'do_sample': False
}

logging.info(f"using textgen params:\n\n:{settings}")

def summarize(ids, mask, **kwargs):
    ids = ids[None, :]
    mask = mask[None, :]

    input_ids = ids.to(_device)
    attention_mask = mask.to(_device)
    global_attention_mask = torch.zeros_like(attention_mask)
    global_attention_mask[:, 0] = 1

    summary_pred_ids = model.generate(
        ids,
        attention_mask=mask,
        global_attention_mask=global_attention_mask,
        return_dict_in_generate=True,
        **kwargs
    )
    summary = tokenizer.batch_decode(
        summary_pred_ids.sequences,
        skip_special_tokens=True,
        remove_invalid_values=True,
    )

    return summary

def summarize_via_tokenbatches(input_text, **kwargs):
    encoded_input = tokenizer(
        input_text,
        padding='max_length',
        truncation=True,
        max_length=token_batch_length,
        stride=batch_stride,
        return_overflowing_tokens=True,
        add_special_tokens=False,
        return_tensors='pt',
    )

    in_id_arr, att_arr = encoded_input.input_ids.to(_device), encoded_input.attention_mask.to(_device)
    gen_summaries = []

    with tqdm(total=len(in_id_arr)) as pbar:
        for _id, _mask in zip(in_id_arr, att_arr):
            result = summarize(ids=_id, mask=_mask, **kwargs)
            _sum = {"input_tokens": _id, "summary": result}
            gen_summaries.append(_sum)
            pbar.update()

    return gen_summaries

# --- Summarization ---

directory = './content'
text_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.txt')]

# Summarize each text file and store in a dictionary
summaries = {} 
for file_path in text_files:
  with open(file_path, 'r', errors='ignore') as f:
    raw_text = f.read()

  long_text = clean(raw_text)
  logging.info(f"removed {len(long_text) - len(raw_text)} chars via cleaning")

  _summaries = summarize_via_tokenbatches(long_text, **settings)
  summaries[file_path] = _summaries  # Store summaries by filename

  if torch.cuda.is_available():
    torch.cuda.empty_cache()

# --- Combine Summaries for Given Author ---

# Assuming you have a DataFrame named 'datf'
def combine_summaries(datf):
  for index, row in datf.iterrows():
    identifiers = row['arxiv-id'].split(',')
    concat_summ = ""

    for identifier in identifiers:
      if identifier.strip() == 'None':
        concat_summ = 'None'
        break

      sanitized_identifier = identifier.strip().replace('/', '_')
      filename = f'SUM_{sanitized_identifier}.txt'  # Assuming summaries are saved as SUM_*.txt

      if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as file:
          concat_summ += file.read()
      else:
        concat_summ = 'File not found'

    datf.at[index, 'Summaries'] = concat_summ

# --- N-grams ---

def sum_n_grams(df, directorypath):
    top10Dict = {'Summaries Top 10 Words': [],
                 'Summaries Top 10 Bigrams': [],
                 'Summaries Top 10 Trigrams': []}

    for i in df.values:
        summaries = i[15]  # Assuming 'Summaries' is the column name for your combined summaries

        # Now use functions from TextAnalysis
        top10words = TA.topwords(summaries, directorypath)
        top10bigrams = TA.topbigrams(summaries, directorypath)
        top10trigrams = TA.toptrigrams(summaries, directorypath)

        top10Dict['Summaries Top 10 Words'].append(top10words)
        top10Dict['Summaries Top 10 Bigrams'].append(top10bigrams)
        top10Dict['Summaries Top 10 Trigrams'].append(top10trigrams)

    sum_top10Df = df
    sum_top10Df['Summaries Top 10 Words'] = top10Dict['Summaries Top 10 Words']
    sum_top10Df['Summaries Top 10 Bigrams'] = top10Dict['Summaries Top 10 Bigrams']
    sum_top10Df['Summaries Top 10 Trigrams'] = top10Dict['Summaries Top 10 Trigrams']

    sum_top10Df = sum_top10Df[['Input Author', 'Input Institution', 'First Author', 'Bibcode', 'Title', 'Publication Date',
             'Keywords', 'Affiliations', 'Abstract', 'Top 10 Words', 'Top 10 Bigrams', 'Top 10 Trigrams', 'Data Type',
            'arxiv-id', 'Summaries', 'Summaries Top 10 Words', 'Summaries Top 10 Bigrams', 'Summaries Top 10 Trigrams']]

    return sum_top10Df

INFO:root:using textgen params:

:{'min_length': 5, 'max_length': 384, 'no_repeat_ngram_size': 7, 'encoder_no_repeat_ngram_size': 7, 'repetition_penalty': 3.7, 'num_beams': 12, 'length_penalty': 0.5, 'early_stopping': True, 'do_sample': False}
INFO:root:removed -221 chars via cleaning


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# # --- Main Execution ---

# # Load your DataFrame
# datf = pd.read_csv('output.csv')  # Replace 'output.csv' with your actual file

# # Combine summaries
# combine_summaries(datf)

# # Calculate N-grams and update DataFrame
# path_stop = '/content/'
# stop_file = 'stopwords.txt'
# stop_dir = path_stop + stop_file
# final_df = sum_n_grams(datf, stop_dir)

# # Save the updated DataFrame
# final_df.to_csv('Results.csv', index=False)
summaries

{'./content\\2410.11851v1.txt': [{'input_tokens': tensor([11900,  7037,    13,  ..., 20992,     9, 15935], device='cuda:0'),
   'summary': ['In this paper, the authors commemorate the half-century anniversary of Stephen hawking\'s discovery of the "Hawking temperature, an equation that predicts that black holes have an absolute temperature proportional to their mass, by calculating the "hooting temperature. This discovery is significant because it revolutionized the field of theoretical physics and black hole physics for the first time. The authors explain how they will derive the hawking temperature using a combination of general relativity, quantum mechanics, and thermodynamics. They will also use dimensional analysis to derive the value of the hawking temperature from the equations used to calculate it. The authors do not yet know the meaning of the hawking temperature, but they plan to explore its possible consequences in the following sections']},
  {'input_tokens': tensor([   4, 